In [4]:
import numpy as np
import random
import math
# Parameters for SHE-COA
NUM_TASKS = 10        # Number of tasks in the system
NUM_FOG_NODES = 5     # Number of available fog computing nodes
NUM_NESTS = 15        # Number of cuckoo nests (solutions)
MAX_ITER = 100        # Maximum iterations
Pa = 0.25             # Abandonment probability
T0 = 100              # Initial temperature (Simulated Annealing)
alpha = 0.95          # Cooling rate for Simulated Annealing

# Simulated energy, latency, bandwidth, and cost matrices
bandwidth = np.random.uniform(10, 100, (NUM_TASKS, NUM_FOG_NODES))
latency = np.random.uniform(1, 10, (NUM_TASKS, NUM_FOG_NODES))
energy = np.random.uniform(5, 50, (NUM_TASKS, NUM_FOG_NODES))
cost = np.random.uniform(1, 20, (NUM_TASKS, NUM_FOG_NODES))

# Fitness Function: Minimize overall cost, energy, latency, and bandwidth
def fitness(solution):
    total_bandwidth = sum(bandwidth[i, solution[i]] for i in range(NUM_TASKS))
    total_latency = sum(latency[i, solution[i]] for i in range(NUM_TASKS))
    total_energy = sum(energy[i, solution[i]] for i in range(NUM_TASKS))
    total_cost = sum(cost[i, solution[i]] for i in range(NUM_TASKS))
    
    weighted_sum=0.3 * total_bandwidth + 0.3 * total_latency + 0.2 * total_energy + 0.2 * total_cost
    # Weighted sum (Adjust weights based on priority)
    return weighted_sum

# Lévy Flight Step (for exploration)
def levy_flight(Lambda=1.5):
    sigma = (math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
             (math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    u = np.random.normal(0, sigma, size=NUM_TASKS)
    v = np.random.normal(0, 1, size=NUM_TASKS)
    step = u / np.abs(v) ** (1 / Lambda)
    return step

# Simulated Annealing Acceptance (for local search)
def simulated_annealing_acceptance(old_fitness, new_fitness, temperature):
    if new_fitness < old_fitness:
        return True
    else:
        prob = np.exp((old_fitness - new_fitness) / temperature)
        return np.random.rand() < prob

# Initialize cuckoo nests (random solutions)
nests = [np.random.randint(0, NUM_FOG_NODES, NUM_TASKS) for _ in range(NUM_NESTS)]
best_nest = min(nests, key=fitness)
best_fitness = fitness(best_nest)

# SHE-COA Algorithm
temperature = T0
for iteration in range(MAX_ITER):
    for i in range(NUM_NESTS):
        # Generate a new solution using Lévy flight
        new_nest = np.copy(nests[i])
        step_size = levy_flight()
        new_nest = (new_nest + step_size).astype(int) % NUM_FOG_NODES  # Ensure valid indices

        # Apply Simulated Annealing to decide acceptance
        if simulated_annealing_acceptance(fitness(nests[i]), fitness(new_nest), temperature):
            nests[i] = new_nest

        # Abandon some nests (exploration)
        if np.random.rand() < Pa:
            nests[i] = np.random.randint(0, NUM_FOG_NODES, NUM_TASKS)

    # Update best nest
    current_best = min(nests, key=fitness)
    current_best_fitness = fitness(current_best)
    
    if current_best_fitness < best_fitness:
        best_nest = current_best
        best_fitness = current_best_fitness

    # Reduce temperature (Simulated Annealing Cooling)
    temperature *= alpha

# Display Final Results
print("✅ Optimized Task Allocation:", best_nest)
print("🎯 Best Fitness Value (Lower is better):", best_fitness)


✅ Optimized Task Allocation: [0 2 4 1 0 1 1 0 0 0]
🎯 Best Fitness Value (Lower is better): 165.4960988068065
